(Введение, о содержании статьи, ссылки на работы)

## Интерпретация моделей машинного обучения

Модели машинного обучения (такие как нейронные сети, машины опорных векторов, ансамбли решающих деревьев) являются "прозрачными" в том смысле, что все происходящие внутри них вычисления известны. Но тем не менее часто говорят, что модели машинного обучения плохо интерпретируемы. Здесь имеется в виду то, что процесс принятия решения не удается представить в понятной человеку форме, то есть:

1. Понять, какие признаки или свойства входных данных влияют на ответ
2. Разложить алгоритм принятия решения на понятные составные части
3. Объяснить смысл промежуточных результатов, если они есть
3. Описать в текстовом виде алгоритм принятия решения (возможно, с привлечением схем или графиков)

Достичь полной интерпретируемости в машинном обучении, как правило, не удается, но даже частичная интерпретация может существенно помочь. Если мы узнаем, на что именно обращает внимание алгоритм, какими правилами руководствуется, то сможем оценить правдоподобность этих правил и тем самым распознать переобучение. Также мы сможем лучше понять, почему модель ошибается на тех или иных примерах, что может помочь улучшить алгоритм обучения.

> Humans usually have prior knowledge about the application domain, which they can use to accept (trust) or reject a prediction if they understand the reasoning behind it. ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$))

Кроме того, интерпретировав алгоритм, мы можем открыть для себя что-то новое о свойствах исследуемых данных (например, какие признаки в табличных данных в наибольшей степени влияют на ответ).

> The need to explain predictions from tree models is widespread. It is particularly important in medical applications, where the patterns uncovered by a model are often even more important than the model’s prediction performance. ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$))

Обзор способов интерпретации моделей машинного обучения можно найти, например, в [Linardatos et al., 2020]($Explainable AI: A Review of Machine Learning Interpretability Methods$) и [Li et al., 2021]($Interpretable Deep Learning: Interpretation, Interpretability, Trustworthiness, and Beyond$). 

### Интерпретация моделей для оценки качества

Авторы методa LIME ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$)) делают акцент на ситуации, когда требуется оценить качество обученной модели (например, чтобы сделать выбор между одной из нескольких моделей). Оценка точности на тестовой выборке не всегда позволяет хорошо оценить качество из-за следующих проблем:

1. <p><b>Сдвиг данных</b> (data shift). Эта ситуация означает, что данные, на которых модель будет применяться, среднестатистически отличаются от тех, на которых модель обучалась и тестировалась, то есть распределение входных данных отличается при тестировании и применении: $P_{test}(X, Y) \neq P_{usage}(X, Y)$. Поскольку $P(X, Y) = P(Y|X)P(X)$, то двумя вариантами сдвига данных является сдвиг в исходных данных $P(X)$ и сдвиг целевой переменной $P(Y|X)$.</p>
<p>Одна из проблем сдвига данных в том, что большинство метрик качества (accuracy, MSE, logloss, F1 и другие) зависят от распределения исходных данных $P(X)$, то есть, упрощенно говоря, от соотношения разных типов примеров в датасете. Например, пусть модель тестировалась на датасете, в котором 80% изображений были высокого качества (HQ), а применяться будет в условиях, когда, наоборот, 80% изображений будут низкого качества (LQ). Пусть мы сравниваем две модели: на HQ-изображениях точность первой модели лучше, чем второй, а на LQ-изображениях, наоборот, точность второй модели лучше, чем первой. Если при тестировании большая часть изображений были HQ, то мы сделаем вывод, что первая модель лучше, тогда как на самом деле лучше была бы вторая.</p>  
<p>Другая проблема сдвига данных в том, что в обучающих и тестовых данных могут присутствовать такие корреляции, которые не обобщаются на другие выборки. Например, мы классифицируем животных по изображению, но большинство изображений рыб, которые у нас имеются, содержат также пальцы рыбака (эти изображения мы используем как при обучении, так и при тестировании). Модель может научиться классифицировать как рыбу изображение, содержащее пальцы, что в целом неверно и может не работать на других выборках (см. также <a href=$Inductive Biases for Deep Learning of Higher-Level Cognition$>Goyal and Bengio, 2020</a> и обзор <a href=$Априорные гипотезы и регуляризация в машинном обучении$>Априорные гипотезы и регуляризация в машинном обучении</a>; пример с рыбами см. в <a href=$Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet$>Brendel and Bethge, 2019</a>).</p>
<p>Сдвиг данных также может быть вызван ошибками (багами) в обработке признаков после развертывания модели. Об этой проблеме мы также поговорим в разделе "Диагностика сдвига данных с помощью SHAP".</p>

2. **Утечка данных** (data leakage, или target leakage). Например, ID пациента может сильно коррелировать с диагнозом, но только в текущем датасете (который поделен на обучающую и тестовую часть). Модель, предсказывающая диагноз по ID, будет иметь высокую точность на тестовом датасете, но в целом очевидно, что в данной задаче такой способ предсказания некорректен и не будет хорошо работать на других данных. Утечка данных является частным случаем сдвига данных, поскольку зависимость ID $\to$ диагноз была в $P_{train}(X, Y)$ и $P_{test}(X, Y)$, но ее не будет в $P_{usage}(X, Y)$. Избавиться от утечки данных не всегда просто.

> Еxample for this is KDD-Cup 2008 breast cancer prediction competition, where the patient ID contained an obvious leak. It is by no means obvious that removing this feature would leave a leakage-free dataset, however. Assuming different ID ranges correspond to different health care facilities (in different geographical locations, with different equipment), there may be additional traces of this in the data. If for instance the imaging equipment's grey scale is slightly different and in particular grey levels are higher in the location with high cancer rate, the model without ID could pick up this leaking signal from the remaining data, and the performance estimate
would still be optimistic (the winners show evidence of this in their report). ([Kaufman et al., 2011]($Leakage in Data Mining: Formulation, Detection, and Avoidance$))

Если удастся интерпретировать модель (хотя бы приблизительно), то мы получим дополнительную информацию, которая поможет надежнее оценить ее качество в условиях возможной утечки и сдвига данных.

### Локальная интерпретация моделей

Вместо попыток интерпретировать модель целиком, что может быть очень сложно, мы можем рассмотреть задачу интерпретации ответа модели $f$ на конкретном, фиксированном примере $x_0$. Например, если на данном изображении модель распознала собаку, то почему она распознала собаку? Какие части и свойства изображения повлияли на предсказание модели?

Для ответа на этот вопрос мы можем изменять $x_0$ и смотреть, как изменится при этом ответ модели, то есть мы изучаем зависимость $f(x_0 + \Delta x)$ от $\Delta x$. При этом возможно удастся с хорошей точностью аппроксимировать эту зависимость простой функцией $g(\Delta x)$. Такой подход называется локальной аппроксимацией модели в окрестности точки $x_0$.

Например, рассчитав градиент $\nabla f(x)$ в точке $x_0$ мы узнаем, как изменится ответ при очень малых изменениях $\Delta x$. При этом мы получаем локальную линейную аппроксимацию (в курсе высшей математики такая аппроксимация называется дифференциалом функции $f$). Такой подход используется, например, при расчете так называемых saliency maps в компьютерном зрении ([Simonyan et al., 2013]($Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps$)) - производных выходных значений сети по отдельным пикселям изображения. Но такая аппроксимация далеко не всегда адекватна:

1. Производная локальна и не говорит о том, как изменится ответ при существенных изменениях $\Delta x$. Например, если один из признаков достиг состояния "насыщения", то есть значение данного признака более чем достаточно, чтобы сделать какой-то вывод о целевой переменной, то производная по нему почти равна нулю. Эффект будет лишь если мы сильно изменим данный признак.
2. В некоторых моделях (решающих деревьях) производная либо равна нулю, либо не существует.
3. Для бинарных признаков производная не всегда информативна, поскольку малое изменение признака ведет в "невозможную" область нецелого значения, в котором модель и не обязана работать корректно.

Есть и другой подход к локальной интерпретации модели: в качестве $\Delta x$ мы можем рассматривать некий набор осмысленных, не бесконечно малых изменений входных данных. Именно такой подход лежит в основе методов интерпретации LIME и SHAP, которые мы рассмотрим далее.

Обзор других методов локальной интерпретации моделей можно найти, например, в [Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$), раздел "Methods 7".

> Local explanations have a distinct advantage over global explanations because by only focusing on a single sample they can remain more faithful to the original model. We anticipate that in the future local explanations will become foundational building blocks for many downstream tasks in machine learning. ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$))

## LIME: Local Interpretable Model-agnostic Explanations

LIME - это подход к интерпретации ответа модели $f(x_0)$ на конкретном тестовом примере $x_0$ с помощью вычисления значений $f(x_0 + \Delta x)$ для некоторого конечного набора значений $\Delta x$. Иллюстрация работы метода LIME приведена на *рис. 1*. Чтобы формально описать метод LIME, нам понадобится ввести ряд обозначений, которые мы будем использовать и в следующих частях обзора.

<img src="assets/lime2.jpg" width="600" align="center">

<center><i>Рис. 1. Иллюстрация работы метода LIME.</i></center>

Мы интерпретируем модель $f$ на примере $x_0$:

- $f: X \to Y$ - исходная модель
- $x_0 \in X$ - выбранный тестовый пример, предсказание на котором $f(x_0)$ интерпретируется

### Локальное упрощенное представление

Для примера $x_0$ вводим $M$ осмысленных, интерпретируемых изменений $\Delta_i$. Например, для изображений таким изменением может быть удаление отдельного суперпикселя, то есть участка изображения с похожим содержимым и четкими границами (*рис. 1b*). Каждое изменение бинарно, то есть оно либо есть, либо нет. Соответственно, мы получаем $2^M$ различных вариантов $\Delta x$. Наличие или отсутствие каждого из изменений можно описать числом 0 или 1: из этих чисел можно собрать бинарный вектор $z$ размерностью $M$, который будем называть *упрощенным представлением* (*рис. 1c*).

- $\{ \Delta_i \}_{i=1}^M$ - изменения примера $x_0$
- $z_i \in \{0, 1\}$ - индикатор изменения $\Delta_i$
- $z \in \{0, 1\}^M$ - вектор упрощенного представления
- $h: \{0, 1\}^M \to X$ - функция, преобразующая вектор упрощенного представления $z$ в $x_0 + \Delta x$

Например, на *рис. 1* функция $h$ работает следующим образом: все "пристствующие" суперпиксели ($z_i = 1$) рисуются без изменений. Все "отсутствующие" суперпиксели ($z_i = 0$) заполняются белым цветом (либо, как вариант, усредненным цветом соседних суперпикселей). При этом $h([1, 1, \dots, 1]) = x_0$, поскольку вектор из единиц означает отсутствие всех изменений. Фактически выбор функции $h$ равносилен выбору изменений $\Delta_i$, то есть семантики упрощенного представления.

Функцию $h$ мы можем выбрать произвольно, но так, чтобы отдельные изменения $\Delta_i$ были интерпретируемы. При этом мы надеемся, что объяснить предсказание модели на $x_0$ можно интерпретировать, изучая, как влияют на ответ эти изменения. Конечно, функция $g$ может быть выбрана неудачно. Например, если модель определяет стиль фотографии как "ретро" при наличии оттенка "сепия", то изменения отдельных суперпикселей не помогут интерпретировать модель. Если $\Delta_i$ были выбраны неудачно, то всегда можно попробовать заново с другими изменениями $\Delta_i$.

Для табличных данных мы можем в качестве $\Delta_i$ рассматривать замену одного из признаков на ноль, среднее или медианное значение по обучающему датасету.

### Объясняющая модель

Теперь мы можем обучить модель $g$ предсказывать значение $f(h(z))$ по вектору упрощенного представления $z$.  При этом модель $g$ должна быть простой и интерпретируемой (поскольку смысл метода LIME в интерпретации). Например, это может быть линейная модель или решающее дерево. Чтобы обучить модель, нужно собрать обучающую выборку. Для этого нам потребуется получить ответ модели $f$ для разных $z$ (то есть для разных $x_0 + \Delta x$, поскольку $z$ определяет $\Delta x$) и таким образом собрать обучающую выборку для модели $g$.

- $g: \{0, 1\}^M \to Y$ - объясняющая модель
- $\big\{ z^{(i)}, f(h(z^{(i)})) \big\}_{i=1}^N$ - обучающая выборка для объясняющей модели

Максимально возможный размер обучающей выборки равен $2^M$, но обычно $M$ велико, и приходится ограничиться перебором лишь некоторых значений $z$. Авторы предлагают уделять основное внимание таким $z$, которые близки к вектору из единиц: это соответствует небольшим изменениям в $x_0$ (чем больше нулей в $z$, тем больше одновременных изменений $x_0$ мы рассматриваем). Введем функцию $\pi(z)$, определяющую меру близости $h(z)$ к $x_0$, и назначим веса $\pi(z^{(i)})$ примерам из обучающей выборки.

- $\pi: \{0, 1\}^M \to \mathbb{R}$ - мера близости $h(z)$ к $x_0$
- $\big\{ w^{(i)} = \pi(z^{(i)}) \big\}_{i=1}^N$ - веса примеров из обучающей выборки

*Примечание.* В kernel SHAP, который мы рассмотрим в следующих разделах, в качестве $\pi$ берется функция, назначающая большие веса как векторам с большим количеством единиц, так и векторам с большим количеством нулей. Так мы акцентируем внимание в том числе на значениях $z$ с большим количеством нулей, то есть на отдельных интерпретируемых компонентах (напрмер, отдельных суперпикселях в случае изображений).

Для обучения модели $g$ осталось выбрать функцию потерь - например, среднеквадратичное отклонение. Эта функция будет сравнивать предсказания моделей $f$ и $g$. Авторы предлагают также использовать "штраф за сложность" $\Omega(g)$ - например, количество ненулевых весов в линейной модели. Если в качестве функции потерь выбрано среднеквадратичное отклонение, то задача оптимизации формулируется следующим образом:

$\sum\limits_{i=1}^N w^{(i)} \Big( g(z^{(i)}) - f(h(z^{(i)})) \Big)^2 + \Omega(g) \to \min\limits_g$

В данной формуле мы считаем квадрат разности предсказаний объясняющей модели $g(z^{(i)})$ и исходной модели $f(h(z^{(i)}))$, и считаем взвешенную сумму по обучающей выборке, используя веса $w^{(i)}$. Кроме того мы прибавляем штраф за сложность объясняющей модели $\Omega(g)$.

### Резюме

Суть подхода LIME в том, что мы аппроксимируем предсказание модели $f$ в окрестности тестового примера $x_0$ более простой, легко интерпретируемой моделью $g$, которая использует упрощенное представление $z$. Например, если модель $g$ линейна, то каждому изменению $\Delta_i$ (например, суперпикселю в изображении) сопоставляется некий вес. 

При этом мы надеемся, что такая аппроксимация адекватна, то есть наличие $i$-го изменения линейно влияет на предсказание модели $f$. В некоторых случаях это может оказаться совсем не так, и модель $g$ не сможет хорошо обучиться (функция потерь остенется высокой). Например, в случае изображений это может означать, что мы не можем линейно влиять на предсказание модели, удаляя отдельные суперпиксели. Возможно, модель ориентируется не на отдельные объекты, а на цвет изображения в целом. Тогда можно попробовать использовать другое упрощенное представление, элементами которого является информация об усредненном цвете изображения.

Работа алгоритма LIME не зависит от вида модели $f$ (нейронная сеть, решающие деревья и т. д.) и никак явно не использует информацию о том, как модель устроена "изнутри", то есть LIME является "model-agnostic" алгоритмом интерпретации.

### Примеры и обсуждение

На *рис. 2* мы видим объяснение предсказания сверточной нейронной сети Inception ([Szegedy et al., 2014]($Going Deeper with Convolutions$)). Сначала мы рассматриваем выходной нейрон, соответствующий классу "Electric guitar", и пытаемся аппроксимировать значение на этом нейроне с помощью линейной модели $g$, которая использует информацию о наличии или отстутствии суперпикселей ($M$ бинарных признаков, где $M$ - количество суперпикселей). В результате для каждого суперпикселя мы получаем вес, то есть вклад этого суперпикселя в предсказание "Electric guitar", и выделяем суперпиксели с наибольшим весом. Далее повторяем тот же алгоритм для двух других выходных нейронов, соответствующих классам "Acoustic guitar" и "Labrador". Как можно видеть из примера, алгоритм LIME концептуально достаточно прост.

<img src="assets/lime.jpg" width="800" align="center">

<center><i>Рис. 2. Пример результатов, полученных с помощью метода LIME. Отмечены суперпиксели с наибольшими весами в линейной объясняющей модели.</i></center>

Работа алгоритма LIME определяется выбором набора изменений $\Delta_i$, этот выбор осуществляется вручную и в каких-то случаях может быть неудачным. Поэтому LIME можно рассматривать как метод, в котором мы сначала формулируем гипотезу о том, как можно было бы объяснить предсказание модели, а затем проверяем ее.

Интересно было бы попробовать применить алгоритм LIME для интерпретации предсказаний человека на различных задачах. Такой подход мог бы дать лучшее понимание плюсов, минусов и границ применимости алгоритма.

Один из возможных минусов заключается в том, что измененные примеры $x_0 + \Delta x$ могут быть неестественными, ненатуральными (например, изображение, в котором стерта часть суперпикселей). Модель $f$, напротив, обучалась только на натуральных примерах, и чаще всего от нее не требуется корректная работа на ненатуральных примерах (не лежащих в многообразии исходных данных, в терминологии из [Fefferman et al., 2013]($Testing the Manifold Hypothesis$)).

Конечно хотелось бы проверять работу $f$ только для тех $x_0 + \Delta x$, которые также выглядят натурально. В табличных данных для более надежной интерпретации можно искать натуральные примеры, похожие на $x_0$ (если датасет имеет достаточный размер) и изучать работу модели на этих примерах (то есть сопоставлять изменение в предсказании модели и изменение входных данных относительно $x_0$.

В компьютерном знении нас может интересовать, каким образом модель предсказывает на изображении собаку. Вместо удаления случайных суперпикселей (что сделает изображение ненатуральным) можно изучить работу модели на видео, в котором переход между кадрами соответствует переходам между разными натуральными изображениями. Можно даже записать видео, по очереди закрывая отдельные части интересующего нас объекта. Также можно попробовать использовать компьютерную графику для генерации натуральных изображений, хотя такой подход сложен и может повлечь много новых проблем.

Конечно, можно разрабатывать более эффективные способы интерпретации для конкретных предметных областей, но особенность LIME именно в том, что это очень общий подход, который может быть применен к широкому классу моделей. При этом многие детали в нем, в частности вид упрощенного представления, могут быть выбраны произвольно.

### LIME-SP для глобальной интерпретации моделей

Авторы также предлагают надстройку над алгоритмом LIME, называемую submodular pick (SP), которая может помочь интерпретировать модель в целом, а не только на конкретном примере. Для этого выбирается набор тестовых примеров, и каждый пример интерпретируется алгоритмом LIME, при этом к каждому примеру мы применяем одни и те же по смыслу изменения. Используя линейную модель $g$, в результате мы получаем матрицу $W$, строкой которой является номер примера, столбцом - позиция в векторе упрощенного представления, значением - вес данной позиции на данном примере.

Например, в случае модели, работающей с текстом и использующей bag-of-words, столбец матрицы $W$ будет соответствовать номер слова в словаре. Однако с суперпикселями так не получится, поскольку на каждом тестовом примере суперпиксели разные по смыслу.

Получив матрицу $W$, мы можем рассчитать глобальный вес каждой позиции, найдя норму каждого столбца. Также мы можем попытаться выбрать небольшой набор строк матрицы $W$ такой, чтобы в этом наборе для каждого столбца хотя бы раз встретилось большое значение. Этот набор строк соответствует набору тестовых примеров, которых предположительно может быть достаточно для глобальной интерпретации модели.

## SHAP: Shapley Additive Explanation Values

В данном разделе мы рассмотрим подход SHAP ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$)), позволяющий оценивать важность признаков в произвольных моделях машинного обучения, а также может быть применен как частный случай метода LIME.

### Shapley values в теории игр

[Теория игр](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%B8%D0%B3%D1%80) - это область математики, изучающей взаимодействие (игру) между игроками, преследующими некие цели и действующими по неким правилам. [Кооперативной игрой](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BE%D0%BF%D0%B5%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%82%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%B8%D0%B3%D1%80) называется такая игра, в которых группа игроков (*коалиция*) действует совместно. С середины XX века ([Shapley, 1952]($A Value for N-Person Games$)) известны так называмые [Shapley values](https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BA%D1%82%D0%BE%D1%80_%D0%A8%D0%B5%D0%BF%D0%BB%D0%B8), которые позволяют численно оценить вклад каждого игрока в достижение общей цели.

Пусть существует *характеристическая функция* (отображение из множества игроков в число), которая описывает эффективность данной коалиции игроков, действующей совместно. Тогда Shapley value для каждого игрока - это число, рассчитываемое по достаточно простой формуле. Обозначим за $\Delta(i, S)$ прирост эффективности от добавления игрока $i$ в коалицию игроков $S$. Пусть всего есть $N$ игроков. Рассмотрим множество $\Pi$ всех возможных упорядочиваний игроков, и обозначим за $(\text{players before } i \text{ in } \pi)$ множество игроков, стоящих перед игроков $i$ в упорядочивании $\pi$. Shapley value для игрока $i$ рассчитывается таким образом:

$\phi(i) = \cfrac{1}{N!} \sum\limits_{\pi \in \Pi} \Delta(i, (\text{players before } i \text{ in } \pi)) \tag{1}$

То есть мы считаем средний прирост эффективности от добавления $i$-го игрока в коалицию игроков, стоящих перед ним, по всем возможным упорядочиваниям игроков (количество элементов суммы равно $N!$). Данная формула задается аксиоматически, то есть формулируется ряд необходимых свойств и доказывается, что существует единственное решение, которое им удовлетворяет. Поскольку $\Delta(i, S)$ не зависит от порядка игроков в $S$, то в формуле $(1)$ можно объединить равные друг другу слагаемые и переписать ее в следующем виде:

$\phi(i) = \sum\limits_{S \subseteq \{1, 2, \dots, N\} \setminus i} \cfrac{|S|! (|N| - |S| - 1)!}{N!}\ \Delta(i, S) \tag{2}$

Данная формула является взвешенной суммой, в которой веса принимают наибольшие значения при $|S| \approx 0$ или $|S| \approx |N|$ и наименьшие значения при $|S| \approx |N|\ /\ 2$.

### Shapley regression values

Тот же подход можно применить в машинном обучении, если игроками считать наличие отдельных признаков, а результатом игры - ответ модели на конкретном примере $x$. Shapley values применяются в машинном обучении еще с XX века ([Kruskal, 1987]($Relative Importance by Averaging Over Orderings$)).

> Game-theoretic formulations of feature importance have become popular as a way to "explain" machine learning models. These methods define a cooperative game between the features of a model and distribute influence among these input elements using some form of the game's unique
Shapley values. ... In this setting, the "players" are the features used by the model, and the game is the prediction of the model. ([Kumar et al., 2020]($Problems with Shapley-value-based explanations as feature importance measures$))

Shapley regression values ([Lipovetsky and Conklin, 2001]($Analysis of Regression in Game Theory Approach$)) позволяют оценить вклад каждого признака в ответ модели $f$. Зафиксируем конкретный тестовый пример $x$, и за $\Delta(i, S)$ будем считать изменение в предсказании $x$ между моделью $f_{S \cup \{i\}}$, обученной на признаках $S \cup \{i\}$ и моделью $f_S$, обученной на признаках $S$:

$\Delta(i, S) = (f_{S \cup \{i\}}(x_{S \cup \{i\}}) - f_S(x_S)) \tag{3}$

Тогда вклад отдельных признаков в величину предсказания модели можно оценивать по формулам $(1)$ и $(2)$. Отметим, что мы рассматриваем не вклад каждого признака в точность модели, а вклад каждого признака в величину предсказания модели на конкретном тестовом примере, что помогает интерпретировать это предсказание. Если модель предназначена для классификации, то в выходными значениями считаются logits - значения до операции softmax/sigmoid. Но можно рассматривать и вклад признаков в величину метрики качества, об этом см. в разделе "Диагностика сдвига данных с помощью SHAP".

В целом есть некоторые проблемы в представлении признака как игрока. Дело в том, что для игрока существует лишь два состояния: либо он есть, либо его нет. Признак же существует как минимум в трех состояниях: два разных значения и неопределенное значение (отсутствие признака). В Shapley regression values сранивается текущее значение признака на примере $x$ с его полным отсутствием при обучении и тестировании.

Минусом такого подхода является высокая сложность вычислений: для расчета Shapley regression values нужно обучать модель на всех возможных подмножествах признаков, что в большинстве случаев невыполнимо. Однако мы можем намного быстрее посчитать приблизительное значение Shapley values, если будем считать не все элементы суммы $(3)$, а лишь некоторые, обладающие большими весами. Для этого мы можем использовать веса как вероятности при семплировании элементов суммы (аппроксимация семплированием, *Shapley sampling values* - [Štrumbelj and Kononenko, 2014]($Explaining prediction models and individual predictions with feature contributions$)).

### SHAP values

Можем ли мы аппроксимировать Shapley regression values, обучая всего одну модель на всех признаках? В этом случае нам нужно получать предсказание модели в случаях, когда многие из признаков имеют неопределенные значения. Большинство моделей не умеют работать с такими данными. Как правило модели машинного обучения моделируют условное мат. ожидание $E [y | x]$, взятое по распределению данных $\mathcal{D}$, иными словами - ожидаемое (среднее) значение $y$ при заданном $x$. Пусть часть признаков известны, часть пропущены. Обозначим за $x_S$ известные признаки, тогда можно считать, что $f(x_S) := E [f(x) | x_S]$. Данная формула означает, что за ответ модели $f$ на примере с пропущенными значениями $x_S$ мы берем мат. ожидание ответа $f$ на примерах $x^\prime$, взятых из распределения данных $\mathcal{D}$, таких, что $x^\prime_S = x_S$. О способе подсчета таких величин мы поговорим далее, но сначала дадим формальное определение SHAP values ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$)).

----

**Определение (SHAP values)**. Пусть мы имеем модель $f$, распределение данных $\mathcal{D}$ и некий тестовый пример $x$ и хотим оценить важность текущих значений каждого признака по сравнению с их неопределенными значениями. SHAP (SHapley Additive exPlanation) values для признаков на примере $x$ - это Shapley values, рассчитываемые для следующей кооперативной игры:

- Игроками являются признаки (наличие $i$-го игрока означает текущее значение $i$-го признака на примере $x$, отсутствие $i$-го игрока означает неопределенное значение $i$-го признака - так же, как в Shapley regression values).
- Характеристической функцией коалиции признаков $S$ является условное мат. ождидание $E [f(x) | x_S]$ по распределению данных $\mathcal{D}$.

----

Таким образом, алгоритм расчета SHAP values следует формулам $(1)$ и $(2)$: для каждого возможного упорядочивания признаков мы берем все признаки, стоящие перед $i$-м признаком (обозначим их за $S$) и считаем величину $\Delta_f(i, S) = E [f(x) | x_{S\ \cup\ i}] - E [f(x) | x_S]$ (о способе подсчета см. далее), после чего усредняем полученные значения по всем упорядочиваниям. Это означает, что SHAP values описывают *ожидаемый прирост выходного значения модели при добавлении $i$-го признака в текущем примере*.

> We propose SHAP values as a unified measure of feature importance. These are the Shapley values of a conditional expectation function of the original model. ...  SHAP values attribute to each feature the change in the expected model prediction when conditioning on that feature. ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$))

Наиболее важным свойством, которым обладают Shapley values, рассчитанные по формулам $(1)$ и $(2)$, является *состоятельность (consistency)*. Это свойство означает, что если (зафиксировав пример $x$) мы рассчитываем Shapley values для двух моделей $f$ и $f^\prime$, и для любых $S$, $i$ верно, что вклад признака $i$ при имеющемся множесте признаков $S$ в первой модели не меньше, чем во второй, то Shapley value этого признака в первой модели также не меньше, чем во второй:

$\textbf{If:}\ \ \forall S, i: \Delta_f(i, S) \geq \Delta_{f^\prime}(i, S) \quad \textbf{Then:}\ \ \phi_f(x) \geq \phi_{f^\prime}(x)$

Отличие SHAP values от Shapley regression values в том, что в последних характеристической функцией группы признаков $x_S$ является значение $f_{x_S}(x_S)$, а в SHAP values - $E [f(x) | x_S]$. В целом эти значения близки, так как $f_{x_S}$ как правило моделирует $E[y|x_S]$. Но Shapley regression values требуют многократного обучения модели и таким образом являются характеристикой обучаемой модели, тогда как SHAP values являются характеристикой обученной модели.

### Проблемы и ограничения SHAP values

На практике рассчет SHAP values позволяет интерпретировать модель, выявлять скрытые проблемы в модели и данных и даже выполнять кластеризацию, что мы увидим далее в разделе "SHAP на практике". Однако стоит помнить о том, что такой подход имеет свои проблемы и ограничения. Проблемы SHAP можно поделить на три класса: вычислительные проблемы, концептуальные проблемы и проблемы ограниченной применимости.

**Вычислительные проблемы**

Определение SHAP не говорит о том, как именно рассчитывать $E [f(x) | x_S]$ в условиях ограниченной по размеру выборки данных. Если бы мы имели прямой доступ к распределению $\mathcal{D}$, неограниченные вычислительные ресурсы и могли бы бесконечно семплировать из $\mathcal{D}$, то была бы возможна оценка $E [f(x) | x_S]$ с любой точностью. Однако обычно мы имеем лишь обучающую и тестовую выборки, взятые из $\mathcal{D}$. Их может быть недостаточно, чтобы надежно оценить $E [f(x) | x_S]$ (см. также [Kumar et al., 2020]($Problems with Shapley-value-based explanations as feature importance measures$), раздел 3.1.1). SHAP values можно искать либо аппроксимацией (Kernel SHAP), либо для конкретных видов моделей (Tree SHAP), что мы рассмотрим в следующих разделах.

**Концептуальные проблемы**

SHAP values основаны на подсчете условного мат. ожидания $E [f(x) | x_S]$, и потому зависят не только от модели $f$, но и от распределения данных. Таким образом, SHAP values являются характеристикой не только модели, а объединенной системы "модель + данные". На разных распределениях данных важность признаков в одной и той же модели может оказаться разной.

Сложно однозначно сказать, является это плюсом или минусом. С одной стороны, существует проблема сдвига данных, когда распределение данных меняется (см. раздел "Интерпретация моделей для оценки качества"). С другой стороны, даже при сдвиге данных существуют совершенно невозможные сочетания признаков, и на таких сочетаниях от модели не требуется корректная работа. За пределами многообразия реальных данных модель может вести себя как угодно, и ее интерпретация в этих областях неинформативна (*рис. 3*).

<img src="assets/manifold.png" width="400" align="center">

<center><i>Рис. 3. Модель может хорошо работать в пределах распределения данных, но вести себя необычно за его пределами. В данном примере один из "хвостов" распределения данных не попал в обучающую выборку из-за ее ограниченного разнообразия, из-за чего модель неверно работает в этой области (проблема сдвига данных).</i></center>

SHAP оценивает *вклад признаков в условное мат. ожидание ответа*, и интерпретировать это как *важность признаков в модели* не всегда корректно. Например, два признака $A$ и $B$ могут сильно коррелировать, при этом модель использует для предсказания только признак $A$. Признак $B$ влияет на мат. ожидание признака $A$, а отсюда и на мат. ожидание ответа модели, следовательно признак $B$ может иметь ненулевое значение SHAP value. Этот признак никак не используется моделью, но он позволяет предсказывать значения других признаков и таким образом влияет на ожидаемый ответ модели.

>  For instance, the popular SHAP algorithm’s formulation may give substantial attributions to features that play no role in the model ([Merrick and Taly, 2019]($The Explanation Game: Explaining Machine Learning Models Using Shapley Values$))

> It is not obvious whether two statistically related features should be considered as separate “players” in the cooperative game, yet this choice has an impact on the output of these additive explanation models. ([Kumar et al., 2020]($Problems with Shapley-value-based explanations as feature importance measures$))

Данная проблема может быть отчасти решена использованием Asymmetric Shapley values (ASVs) ([Frye et al., 2019]($Asymmetric Shapley values: incorporating causal knowledge into model-agnostic explainability$)), если коррелирующие переменные связаны причинно-следственной связью и эта связь известна заранее. В этом обзоре мы не будем разбирать этот метод.

**Проблемы ограниченной применимости**

Shapley values являются достаточно гибким инструментом, который можно применять во многих случаях для оценки вклада "участников процесса" в результат. Но SHAP values не являются универсальным способом интерпретации как минимум потому, что выходные данные могут иметь сложный формат, а входные признаки могут быть не интерпретируемы.

### Линеная аппроксимация SHAP

Задачу оценки $E [f(x) | x_S]$, можно сильно упростить, если [предположить](https://en.wikipedia.org/wiki/Statistical_assumption), что наличие каждого признака линейно и независимо влияет на ответ модели. Тогда значения признаков, не входящих в $S$, не зависят от $x_S$ и друг от друга и линейно влияют на ответ, и $E [f(x) | x_S]$ можно рассчитать, заменив пропущенные значения их мат. ожиданиями, которые можно приблизить средним значением по выборке данных. Обозначив пропущенные признаки за $\overline{S}$, получим

$f(x_S) := E [f(x) | x_S] \approx f([x_S, E[x_\overline{S}]])$

> feature independence and model linearity are two optional assumptions simplifying the computation of the expected values ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$))

В формуле стоит знак приблизительного равенства потому, что линейность и независимость - это лишь предположения: чем ближе они к истине, тем точнее аппроксимация. С помощью данной формулы мы можем рассчитать $f(x_S)$ для любого подмножества признаков $x_S$, а значит получили возможность рассчитывать SHAP values на практике по формуле $(2)$, опционально применяя также аппроксимацию семплированием.

Не является ли линейность и независимость влияния признаков чрезмерно грубым упрощением? В некоторых случаях возможно является, но вспомним, что в методе LIME мы использовали такое же допущение, обучая линейную объясняющую модель $g$. О связи SHAP и LIME мы поговорим далее.

### Kernel SHAP

Авторы сопоставляют SHAP с несколькими появившимися в 2010-х годах методами интерпретации моделей машинного обучения, которые используют локальную аппроксимацию, в первую очередь с LIME, а также с *layer-wise relevance propagation* ([Bach et al., 2015]($On Pixel-Wise Explanations for Non-Linear Classifier Decisions by Layer-Wise Relevance Propagation$)) и *activation difference propagation (DeepLIFT)* ([Shrikumar et al., 2017]($Learning Important Features Through Propagating Activation Differences$)), которые мы сейчас не будем рассматривать.

Как LIME, так и DeepLIFT рассматривают локальное упрощенное представление в виде вектора $z$ (где $z_i = 0$ означает наличие изменения $\Delta_i$ относительно примера $x_0$). В разделе, посвященном LIME, мы говорили о таком представлении и об объясняющей модели $g$, аппроксимирующей $f(h(z))$. Авторы SHAP рассматривают случай, когда объясняющая модель линейна (такие способы интерпретации авторы называют additive feature attribution methods):

$g(z) = \phi_0 + \sum\limits_{i=1}^M \phi_i z_i, \quad \phi_i \in \mathbb{R}, z_i \in \{0, 1\} \tag{4}$

Важным является случай, когда $\Delta_i$ означает удаление некоего признака из $x_0$, то есть при $z_i = 0$ один из признаков в $x_0$ заменяется на неопределенное значение. В этом случае будем говорить, что $z_i$ *задает наличие признака*. При этом постановка задачи становится эквивалентой той, на которой основан SHAP. Если при этом модель $g$ линейна, то мы можем использовать linear SHAP values, то есть заменять отсутствующие признаки их средним значением.

Вспомним, что в LIME при выборе линейной модели $g$ мы затем искали ее веса $\phi_i$, но для этого мы должны были выбрать функцию потерь $\mathcal{L}$, метрику сходства $\pi$ и штраф сложности $\Omega$. Авторы SHAP выводят единственно возможные значения для $\mathcal{L}$, $\pi$ и $\Omega$ такие, что полученные веса $\phi_i$ будут равны SHAP values, рассчитанным по формуле $(2)$, при обучении на всех возможных $z$. Если же мы будем обучать $g$ не на всех $z$, то полученные веса будут аппроксимировать $\phi_i(f, x_0)$.

- $\mathcal{L}$ - среднеквадратичное отклонение
- $\Omega(g) = 0$
- $\pi(z) = \cfrac{M-1}{{{M}\choose{|z|}}\ |z|\ (M-|z|)}$, где ${M}\choose{|z|}$ - [биномиальный коэффициент](https://en.wikipedia.org/wiki/Binomial_coefficient)

Основное отличие от примера, предложенного авторами LIME ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$)), состоит именно в $\pi(z)$. Как мы видели в разделе "Объясняющая модель", $\pi(z)$ фактически является метрикой сходства между векторами, такие метрики часто называют ядрами (kernels), поэтому предложенная формула для $\pi(z)$ получила название **Shapley kernel**, а метод назван **kernel SHAP**.

Shapley kernel назначает большие веса примерам как с большим количеством единиц, так и с большим количеством нулей.  Это означает, что мы обращаем основное внимание на примеры с (почти) максимальным и (почти) минимальным количеством компонентов (игроков). На *рис. 4* показано отличие Shapley kernel от L2 distance и cosine similarity, которые предлагалось использовать в LIME.

<img src="assets/shap.jpg" width="400" align="center">

<center><i>Рис. 4. Shapley kernel в сравнении с L2 distance и cosine similarity.</i></center>

### Tree SHAP

SHAP values основаны на условном мат. ождидании $E [f(x) | x_S]$, но не всегда понятно, как посчитать это значение (см. раздел "SHAP values"). Модели, основанные на решающих деревьях (градиентный бустинг, случайный лес) позволяют работать с произвольным количеством пропусков в данных, поэтому в таких моделях можно напрямую рассчитать значение $E [f(x) | x_S]$, сделав предсказание на признаках $x_S$.

Получить предсказание решающего дерева для примера с пропущенными значениями можно следующим образом. Рассмотрим пример $x$ и дерево $D$, которое начинается разделяющим правилом, ведущим к двум поддеревьям $D_1$ и $D_2$. Если данное правило основано на признаке, который известен, тогда на основе значения признака мы выберем нужное поддерево $D_i(x)$ и возьмем его ответ: $D(x) = D_i(x)$. Если же признак пропущен, то в качестве ответа возьмем величину $D(x) = (c_1 D_1(x) + c_2 D_2(x))$, где $c_1$ и $c_2$ - доли обучающих примеров, которые попали в первое и во второе дерево. Таким образом мы аппроксимируем значение $E [f(x) | x_S]$ с помощью обучающей выборки.

SHAP values можно посчитать по формуле $(2)$, но количество слагаемых экспоненциально зависит от количества признаков, и алгоритм рассчета SHAP values в общем случае является NP-полным. Однако если ограничиться только решающими деревьями, то для расчета формулы $(2)$ существует алгоритм c полиномиальной сложностью низкого порядка, предложенный в [Lundberg et al., 2018]($Consistent Individualized Feature Attribution for Tree Ensembles$) и реализованный в python-библиотеке [shap](https://github.com/slundberg/shap).

## SHAP на практике

Для подсчета SHAP values существует python-библиотека [shap](https://github.com/slundberg/shap), которая может работать со многими ML-моделями (XGBoost, CatBoost, TensorFlow, scikit-learn и др) и имеет документацию с большим количеством примеров. С помощью бибилиотеки SHAP можно строить различные схемы и графики, описывающие важность признаков в модели и их влияние на ответ. Рассмотрим примеры из документации.

### Waterfall plot

На *рис. 5* показан [waterfall](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/waterfall.html) plot, объясняющий предсказание на первом тестовом примере из датасета Boston housing. SHAP values получены с помощью метода Tree SHAP. Схема читается снизу вверх, и признаки упорядочены по возрастанию их SHAP values. Например, SHAP value -0.43 для признака CRIM (имеющего значение 0.006) говорит о том, что значение CRIM=0.006 на данном примере уменьшает величину предсказания модели, по сравнению с отсутствием признака CRIM, при произвольном наличии других признаков (см. "Shapley values в теории игр").

<img src="assets/shap2.png" width="450" align="center">

<center><i>Рис. 5. Waterfall plot.</i></center>

### Summary plot

Рассчитав SHAP value для каждого признака на каждом примере (с помощью kernel SHAP или tree SHAP), мы можем построить summary plot, то есть summary plot объединяет информацию из waterfall plots для всех примеров.

На *рис. 5* summary plot построен для модели, обученной на датасете, описывающем влияние различных медицинских анализов на вероятность смерти в течение следущих 12 лет ([Cox et al., 1997]($Plan and operation of the NHANES I Epidemiologic Followup Study, 1992$)). Рисунок взят из [Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$). SHAP values при этом получены с помощью метода Tree SHAP.

Каждая горизонтальная линия соответствует одному признаку, и на этой линии отмечаются точки, соответствующие тестовым примерам: кордината точки на линии соответствует SHAP value, цвет точки - значению признака. Если в каком-то участке линии не хватает места для всех точек, линия начинает расти в ширину. Таким образом, для каждого признака схема представляет собой слившееся множество точек, по одной точке для каждого примера

Слева на *рис. 6* показана важность признаков, рассчитанная как средний модуль величины SHAP values для данного признака.

<img src="assets/shap3.jpg" width="800" align="center">

<center><i>Рис. 6. Summary plot.</i></center>

Очевидное влияние на риск смерти имеет возраст, а также пол (женщины среднестатистически живут дольше). Мы видим, что чем больше возраст (красный цвет), тем большее SHAP value (горизонтальная ось) назначается этому признаку. Большое значение SHAP value, в свою очередь, означает, что удаление этого признака (замена значения возраста на неопределенное) существенно уменьшит предсказанную вероятность смерти в течение 12 лет.

Можно также заметить, что большинство SHAP values, соответствующих медицинским анализам (строки 3 и далее), имеют либо оклонулевые, либо положительные значения SHAP values. Это означает, что есть много значений анализов, увеличивающих предполагаемый риск смерти (для данной модели), но нет таких значений, которые бы его сильно уменьшали.

> Interestingly, rare mortality effects always stretch to the right, which implies there are many ways to die abnormally early when medical measurements are out-of-range, but not many ways to live abnormally longer. ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$))

Одно из преимуществ SHAP summary plot по сравнению с глобальными методами оценки важности признаков (такими, как mean impurity decrease или permutation importance) состоит в том, что на SHAP summary plot можно различить 2 случая: (А) признак имеет слабое влияние, но во многих примерах, (Б) признак имеет большое влияние, но в немногих примерах.

> SHAP summary plots avoid conflating the magnitude and prevalence of an effect into a single number, and so reveal
rare high magnitude effects. ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$))

### Dependence plot

На *рис. 7C* мы видим совместное распределение SHAP value для признака *Systolic blood pressure* и значения этого признака по тестовому датасету. Фактически это та же самая информация, что показана в summary plot для этого признака. На *рис. 7D* показаны SHAP interaction values (их способ рассчета мы не рассматривали в этом обзоре) для пары признаков: *Systolic blood pressure* + *Age*. На *рис. 7B* показана схема (SHAP dependence plot), объединяющая информацию из схем на *рис. 7C и 7D*. Рисунки взяты из [Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$).

Как можно видеть, совместное распределение на *рис. 7B* имеет "два хвоста". Для возраста 30-40 (синий цвет) систолическое давление 160+ mmHg повышает предсказанную вероятность смерти в течение 12 лет, тогда как для возраста 60-70+ это же давление является нормальным (соответствующим возрасту) и дополнительно не повышает вероятность смерти в течение 12 лет.

> early onset high blood pressure is more concerning to the model than late onset high blood pressure ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$))

Следует помнить, что при этом мы изучаем обученную модель, а не только данные. Модель может ошибаться, и тогда выводы о целевой зависимости, сделанные на основе SHAP values, тоже будут неверны.

<img src="assets/shap4.jpg" width="1200" align="center">

<center><i>Рис. 7. Dependence plot.</i></center>

### Диагностика сдвига данных с помощью SHAP loss values

Как мы говорили в разделе "Интерпретация моделей для оценки качества", сдвиг данных является серьезной проблемой в машинном обучении. Он может быть вызван недостаточным разнообразием обучающих данных, изменениями в методах расчета признаков и ошибками в их обработке после развертывания модели.

> Deploying machine learning models in practice is challenging because of the potential for input features to change after deployment. It is hard to detect when such changes occur, so many bugs in machine learning pipelines go undetected, even in core software at top tech companies. ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$)).

Для диагностики сдвига данных можно использовать Shapley values, считая результатом кооперативной игры не предсказание модели, а функцию потерь. Тогда мы сможем определить, какие признаки вносят положительный или отрицательный вклад в точность модели.

В работе "Explainable AI for Trees" ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$)) авторы проводят следующий эксперимент. В качестве данных используется датасет, целевым признаком в котором является длительность процедуры анастезии перед операцией. Датасет содержит данные за 4 года (2185 признаков и ок. 147000 пациентов). Данные за первый год используются для обучения, данные за следующие три года - для тестирования (симуляции работы системы по разначению). Отметим, что это не временной ряд: каждый пример независим. 

График метрики качества на тестовых данных (в зависимости от даты) показан на *рис. 8*. Такие графики используют для мониторинга качества работы системы. Метрика качества на обучающих данных (первый год) намного лучше, что естественно.

<img src="assets/shap5.jpg" width="700" align="center">

<center><i>Рис. 8. Метрика качества на обучающих и тестовых данных в датасете "hospital procedure duration".</i></center>

На каждом тестовом примере можно рассчитать SHAP loss values, считая игроками наличие признаков, а характеристической функцией набора признаков - точность предсказания на данном примере. Поскольку каждый тестовый пример соответствует определенной дате, то мы можем построить для каждого признака график зависимости Shapley values от даты. На *Рис. 9* показаны такие графики для трех бинарных признаков (цветом обозначено значение признака).

<img src="assets/shap6.jpg" width="700" align="center">

<center><i>Рис. 9. Мониторинг SHAP values обнаруживает проблемы, не видимые по графику метрики качества, в датасете "hospital procedure duration".</i></center>

Первый признак *"in room #6"* означает, что процедура проводилась в комнате номер 6. При обучении этот признак понижал ошибку предсказания времени процедуры, но при симуляции работы по назначению с какого-то момента стал резко повышать ошибку, то есть наличие этого признака стало вредным. Причина кроется в том, что в коде обработки признаков была допущена ошибка: перепутаны номера комнат 6 и 13. Данная ошибка была допущена исследователями намеренно, чтобы показать эффективность SHAP в обнаружении подобных проблем. При этом на графике функции потерь этой проблемы не видно, так как она затрагивает лишь небольшой процент данных.

Следующие две проблемы, напротив, были ненамеренными и обнаружены случайно. Признак *"general anasthesia"* в определенный момент резко снижал точность предсказания. Как выяснилось, проблема была связана с ошибкой в конфигурации электронного оборудования.

На признаке *"atrial fibrillation"* виден дрейф значений SHAP loss values: со временем этот признак становится все менее полезным, и в итоге начинает понижать точность предсказания времени процедуры. Как выяснилось, это вызвано изменениями в длительности процедуры абляции фибрилляции предсердий, которая связана с изменениями в технологиях и персонале.

### Supervised-кластеризация данных с помощью SHAP

Из SHAP values можно составить SHAP-вектор для каждого обучающего или тестового примера. [Lundberg et al., 2018]($Consistent Individualized Feature Attribution for Tree Ensembles$) заметили, что SHAP-векторы можно использовать для кластеризации данных намного эффективнее, чем вектора исходных признаков.

Задача кластеризации данных достаточно нетривиальна. Как правило, мы не можем выполнить кластеризацию просто на основе евклидова расстояния между векторами признаков, так как признаки различаются по важности, имеют разный масштаб, и некоторые признаки могут дублировать друг друга.

Элементы SHAP-вектора соответствуют отдельным признакам, но при этом каждый элемент SHAP-вектора представлен в одной и той же шкале. При этом величина SHAP value означает важность признака (в контексте задачи предсказания целевого признака). Исходя из этого, можно выполнять supervised-кластеризацию в пространстве SHAP-векторов.

> The distance metric does not account for the discrepancies among the units/meaning of features (e.g., weight vs. age), and there
is no way for an unsupervised approach to know which features are relevant for an outcome of interest, and so should be weighted more strongly. We can address both of these limitations by using *local explanation embeddings* to embed each sample into a new "explanation space." If we run clustering in this new space, we will get a *supervised clustering* where samples are grouped together based on their *explanations*. Supervised clustering naturally accounts for the differing units of various features, only highlighting changes that are relevant to a particular outcome. ...  Analogously, we can also run PCA on local explanation embeddings... ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$))

На *рис. 10* показана кластеризация с помощью SHAP на датасете, описывающем риск смерти в течение 12 лет на основе медицинских анализов (об этом датасете см. также раздел "Summary plot"). Каждый столбец представляет собой SHAP-вектор для одного пациента, и эти векторы упорядочены (локально) по сходству друг с другом с помощью алгомеративной иерархической кластеризации и (глобально) по значению целевого признака, объединяясь в кластеры людей со сходной симптоматикой.

<img src="assets/shap7.jpg" width="900" align="center">

<center><i>Рис. 10. Кластеризация данных с помощью SHAP values.</i></center>

Другой пример из [Lundberg et al., 2018]($Consistent Individualized Feature Attribution for Tree Ensembles$) изображен в трехмерном формате (*рис. 11*). В данном случае показаны примеры из датасета, в котором целевой переменной является вероятность месячного заработка больше \$50К. Примеры здесь тоже кластеризованы с помощью их SHAP-векторов ("sorted by explanation similarity"). Полученным кластерам даны текстовые описания.

<img src="assets/shap8.jpg" width="900" align="center">

<center><i>Рис. 12. Еще один пример кластеризации данных с помощью SHAP values.</i></center>